<a href="https://colab.research.google.com/github/ZeddBR/projeto-aula/blob/main/Music_Updater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [8]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="agente que busca informacoes no Google",
        tools=[google_search],
        instruction="""
        Voce é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para realizar uma busca geral da história, novidades e últimos lançamentos musicais de bandas ou artistas da música mundial.
        Buscar novidades e noticias sobre as bandas e citar singles ou clipes lançados na última semana ou de uma banda especificada no topico.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [50]:
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Voce é um redator de conteudo musical, especialista em músicas e bandas de rock do estilo emocore, emo, post-emo, midwest emo.
        Com base na lista de lancamentos mais recentes e relevantes buscados, voce deve:
        usar a ferramenta de busca do google (google_search) para criar uma lista detalhada, e atual,
        listando novidades, nome da faixa, artista, data de lançamento, nome do álbum se houver conforme informacoes recebidas do agente anterior.
        Liste também as redes sociais do artista e as plataformas disponíveis para escutar a música ou assistir ao clipe com links de redirecionamento.
        Voce tambem pode usar o (google_search) para encontrar informacoes sobre os temas e aprofundar.
        """,
        description="Agente que gera a lista e conteudos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    lista_top_10 = call_agent(planejador, entrada_do_agente_planejador)
    return lista_top_10

In [55]:
def agente_redator(topico, lista_top_10):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado listar de forma organizada o conteudo gerado pelo agente planejador, sem repetir informações.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado. Verifique se os links estão funcionais antes de envia-los.
            Gere a lista sobre o tópico indicado, liste organizando de forma clara nome do artista do topico, da música ou faixa, album, ano de lançamento e links
            das redes sociais e das plataformas de streaming disponíveis.
            Utilize fonte em negrito para títulos. Se possível, inclua uma foto da capa do album ou do artista/banda.
            Apenas liste, sem comentários adicionais, de forma bem organizada e resumida com os links, agenda de shows a partir de hoje e uma lista de até 10 músicas mais escutadas do artista do topico.
            """,
        description="Agente redator e do post final."

    )

    entrada_do_agente_redator = f"Tópico: {topico}\nRascunho: {lista_top_10}"
    lista_final = call_agent(redator, entrada_do_agente_redator)
    return lista_final

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🎧 Buscando novidades do seu artista favorito para você! 🎧")

topico = input("❓ Por favor, digite o nome do artista ou banda que deseja receber as últimas atualizações: ")

if not topico:
  print("Voce esqueceu de digitar o nome do artista/banda")
else:
    print(f"Pronto! Agora se liga nas novidades de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Busca Principal ---\n")
    display(to_markdown(lancamentos_buscados))
    print("-------------------------------------------------------")

    lista_top_10 = agente_planejador(topico, lancamentos_buscados)
    print("\n--- Novidades! ---\n")
    display(to_markdown(lista_top_10))
    print("-------------------------------------------------------")

    lista_final = agente_redator(topico, lista_top_10)
    print("\n--- Links e atualizações ---\n")
    display(to_markdown(lista_final))
    print("-------------------------------------------------------")